# llama2

In [ ]:
#!huggingface-cli login

In [6]:
import os
from torch import nn
import torch, transformers
os.environ["CUDA_VISIBLE_DEVICES"]="2"

SyntaxError: unterminated string literal (detected at line 1) (2580916956.py, line 1)

In [ ]:
import os
from torch import nn
import torch, transformers
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from transformers import LlamaForCausalLM, LlamaTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(model_name)
model =LlamaForCausalLM.from_pretrained(model_name,
                                        load_in_8bit=True,
                                        device_map="auto" # gpu 꽉차면 cpu로 올려줌 
                                       )


In [7]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSN

In [209]:
(torch.tensor([[3,4]]),)+(torch.tensor([[3,4]]),)

(tensor([[3, 4]]),)

In [210]:
(torch.tensor([[3,4]]),)+(torch.tensor([[3,4]]),)

(tensor([[3, 4]]), tensor([[3, 4]]))

In [150]:
from transformers import LlamaConfig
LlamaConfig()

LlamaConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [210]:
prompt = "I am studying" 
output_so_far = torch.tensor([tokenizer.encode(prompt)])
print(output_so_far, output_so_far.shape)

tensor([[    1,   306,   626, 23382]]) torch.Size([1, 4])


In [173]:
lm_output = model(input_ids = output_so_far)
print(lm_output.keys())

odict_keys(['logits', 'past_key_values'])


In [177]:
past = lm_output['past_key_values']
len(past) , len(past[0]),past[0][0].shape
# layer 수, key_value 2개, (batch_size,num_heads,sequence_length, embed_size_per_head)

(32, 2, torch.Size([1, 32, 4, 128]))

In [8]:
logits = lm_output['logits'] # torch.Size([1, 4, 32000])

probs = nn.functional.softmax(logits[:,-1,:], dim=-1) # torch.Size([1, 32000])
last = torch.topk(probs, k=1)[1]  # tensor([[363]]) 

NameError: name 'lm_output' is not defined

In [168]:
output_so_far = torch.cat((output_so_far,last),dim=1)
print(tokenizer.decode(output_so_far[0]))

<s>I am studying for


In [169]:
lm_output = model(input_ids = output_so_far)
logits = lm_output['logits'] 

probs = nn.functional.softmax(logits[:,-1,:], dim=-1) 
last = torch.topk(probs, k=1)[1]

output_so_far = torch.cat((output_so_far,last),dim=1)

tensor([[    1,   306,   626, 23382,   363]])

In [170]:
lm_output = model(input_ids = output_so_far)
logits = lm_output['logits']
probs = nn.functional.softmax(logits[:,-1,:], dim=-1) # torch.Size([1, 32000])
last = torch.topk(probs, k=1)[1]  # tensor([[363]])
output_so_far = torch.cat((output_so_far,last),dim=1)
print(tokenizer.decode(output_so_far[0]))

<s>I am studying for my


In [272]:
lm_output = model(input_ids = output_so_far,
          output_hidden_states=True,
          output_attentions = True,
                 use_cache = True )

logits = lm_output['logits']
logits[0][0]

tensor([-13.0703,  -6.2422,  -0.0146,  ...,  -6.8711,  -8.1250,  -7.6172])

In [274]:
past = lm_output['past_key_values']
past

((tensor([[[[-0.4446, -0.0028,  0.0523,  ...,  0.0536, -0.0229, -0.1245],
            [-0.0420, -0.2524,  0.0391,  ...,  0.1910, -0.0818,  0.2451],
            [ 0.3093, -0.2466, -0.4319,  ..., -0.0919,  0.3347,  0.1709],
            [ 0.0481, -0.0416, -0.1141,  ...,  0.5059, -0.2888,  0.4900]],
  
           [[ 1.3389,  1.0771, -0.4302,  ...,  0.4849, -0.3022,  0.4773],
            [-0.0815,  0.3335,  0.2727,  ..., -0.6724,  0.3066, -0.5234],
            [-0.6074, -0.1863, -0.2119,  ..., -0.4590,  0.4514, -0.4116],
            [-0.8389, -0.2351,  0.5171,  ..., -0.0099,  0.2944, -0.0473]],
  
           [[ 0.0051, -0.2598, -0.4148,  ..., -0.1182,  0.5000,  0.7603],
            [ 0.1710,  0.2756,  0.2236,  ...,  1.7910,  1.8652,  1.7109],
            [ 0.8149,  0.9507,  0.1338,  ...,  0.6289,  0.7012,  0.6660],
            [ 0.0181,  0.4878,  0.2441,  ..., -0.5703, -0.6172, -0.5337]],
  
           ...,
  
           [[ 0.0291, -0.0068,  0.0407,  ...,  0.4871,  0.9614, -0.3987],
       

In [212]:
print(output_so_far)

tensor([[    1,   306,   626, 23382]])


In [263]:
lm_output = model(input_ids = output_so_far,
                  use_cache = False,
                  output_hidden_states=True,
                  output_attentions = True,
                  attention_mask = torch.tensor([[1,1,0,0]]))
lm_output.keys()

odict_keys(['logits', 'hidden_states', 'attentions'])

In [264]:
attentions = lm_output['attentions']
print(len(attentions),attentions[0].shape)
# layer수, (batch_size, num_heads, sequence_length, sequence_length).

32 torch.Size([1, 32, 4, 4])


In [265]:
attentions

(tensor([[[[1.0000, 0.0000, 0.0000, 0.0000],
           [0.8779, 0.1221, 0.0000, 0.0000],
           [0.8457, 0.1543, 0.0000, 0.0000],
           [0.6602, 0.3401, 0.0000, 0.0000]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.6929, 0.3074, 0.0000, 0.0000],
           [0.5610, 0.4392, 0.0000, 0.0000],
           [0.6440, 0.3557, 0.0000, 0.0000]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.8657, 0.1344, 0.0000, 0.0000],
           [0.8354, 0.1643, 0.0000, 0.0000],
           [0.6519, 0.3479, 0.0000, 0.0000]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.6855, 0.3145, 0.0000, 0.0000],
           [0.6938, 0.3062, 0.0000, 0.0000],
           [0.7524, 0.2477, 0.0000, 0.0000]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.0258, 0.9741, 0.0000, 0.0000],
           [0.0301, 0.9697, 0.0000, 0.0000],
           [0.0688, 0.9312, 0.0000, 0.0000]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.2637, 0.7363, 0.0000, 0.00

In [235]:
hidden_states = lm_output['hidden_states']
len(hidden_states),hidden_states[0].shape
# 1+layer수,(batch_size, sequence_length,embedding_size)

(33, torch.Size([1, 4, 4096]))

1 for tokens that are not masked,

0 for tokens that are masked

default =1 

In [260]:
128/4

32.0

In [261]:
lm_output = model(input_ids = output_so_far,
                  use_cache = False,
                  attention_mask = torch.tensor([[1,1,1,1]]),
                  output_hidden_states=True,
                  output_attentions = True)
lm_output.keys()
attentions = lm_output['attentions']
print(len(attentions),attentions[0].shape) 
# layer수, (batch_size, num_heads, sequence_length, sequence_length).
attentions

32 torch.Size([1, 32, 4, 4])


(tensor([[[[1.0000, 0.0000, 0.0000, 0.0000],
           [0.8779, 0.1221, 0.0000, 0.0000],
           [0.6206, 0.1133, 0.2659, 0.0000],
           [0.2585, 0.1333, 0.5522, 0.0557]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.6929, 0.3074, 0.0000, 0.0000],
           [0.4019, 0.3147, 0.2834, 0.0000],
           [0.3616, 0.1997, 0.1882, 0.2505]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.8657, 0.1344, 0.0000, 0.0000],
           [0.7104, 0.1398, 0.1498, 0.0000],
           [0.4282, 0.2286, 0.1680, 0.1750]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.6855, 0.3145, 0.0000, 0.0000],
           [0.6172, 0.2722, 0.1108, 0.0000],
           [0.5874, 0.1934, 0.1096, 0.1096]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.0258, 0.9741, 0.0000, 0.0000],
           [0.0261, 0.8428, 0.1311, 0.0000],
           [0.0078, 0.1060, 0.0188, 0.8672]],
 
          [[1.0000, 0.0000, 0.0000, 0.0000],
           [0.2637, 0.7363, 0.0000, 0.00

In [119]:
logits = lm_output['logits']

past_key_values = lm_output['past_key_values']
hidden_states = lm_output['hidden_states']

KeyError: 'hidden_states'

In [121]:
past_key_values[0][0].shape

torch.Size([1, 32, 4, 128])

In [87]:
logits.shape

torch.Size([1, 4, 32000])

In [47]:
z['hidden_states'][-1].shape

torch.Size([1, 4, 4096])

In [34]:
last_hidden_state = lm_output['hidden_states'][-1]

In [52]:
last_hidden_state[:,-1,:]

torch.Size([1, 4096])

In [35]:
model.lm_head(last_hidden_state)

tensor([[[-13.1328,  -5.7148,  -0.0936,  ...,  -6.8789,  -8.1172,  -7.7031],
         [ -9.5391, -12.2344,  -1.0869,  ...,  -5.0508,  -8.9297,  -3.6543],
         [ -7.1406,  -4.6406,   2.1484,  ...,  -1.9463,  -5.7695,  -1.9766],
         [-10.4297,  -8.1719,  -0.7935,  ...,  -8.1016,  -8.7656,  -6.9531]]],
       device='cuda:0', dtype=torch.float16)

In [36]:
logits.shape

torch.Size([1, 4, 32000])

In [37]:
logits[:,-1,:]

tensor([[-10.4297,  -8.1719,  -0.7935,  ...,  -8.1016,  -8.7656,  -6.9531]])

In [38]:
from torch import nn
probs = nn.functional.softmax(logits[:,-1,:], dim=-1)
probs.shape

torch.Size([1, 32000])

In [39]:
last = torch.topk(probs, k=1)[1]
last

tensor([[363]])

In [40]:
output_so_far = torch.cat((input_ids,last),dim=1)

In [41]:
output_so_far = torch.cat((input_ids,last),dim=1)
print(tokenizer.decode(output_so_far.tolist()[0]))

<s>I am studying for


## past이용

In [195]:
output_so_far

tensor([[    1,   306,   626, 23382,   278,   278,   278]])

In [189]:
last = output_so_far[:, -1:] # 마지막 단어
past = model(output_so_far[:, :-1])["past_key_values"] #이전단어들의 key_value

In [197]:
lm_output= model(last, 
                 past_key_values= past)  

logit = lm_output["logits"]   
past  = lm_output["past_key_values"] # past 업데이트 

probs = nn.functional.softmax(logit[:,-1,:], dim=-1) 
last = torch.topk(probs, k=1)[1] # last 업데이트

In [194]:
output_so_far = torch.cat((output_so_far,last),dim=1)
print(tokenizer.decode(output_so_far[0]))

<s>I am studying the the the


In [198]:
len(past),len(past[0]),past[0][0].shape

(32, 2, torch.Size([1, 32, 5, 128]))

In [49]:
lm_output["hidden_states"][-1]

tensor([[[-1.1729, -1.6465,  1.8525,  ..., -1.3242,  0.7266,  0.8311]]],
       dtype=torch.float16)

# huggingface generate 함수

In [109]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSN

In [277]:
prompt1 = "I am studying"

encoded_prompt1 = tokenizer.encode(prompt1, return_tensors="pt")
print("input: ", encoded_prompt1)
print("-----------------------------------------------")

# Generate
generated_ids = model.generate(input_ids = encoded_prompt1, 
                               max_length=14,
                               attention_mask = torch.tensor([[1,1,1,1]]),
                               use_cache = False,
                  output_hidden_states=True,
                  output_attentions = True,
                              )
print("output: ", generated_ids)

generated_sentence = tokenizer.decode(generated_ids[0])
print(generated_sentence)


input:  tensor([[    1,   306,   626, 23382]])
-----------------------------------------------
output:  tensor([[    1,   306,   626, 23382,   363,   590,  9082, 29915, 29879,  7426,
           297,   278, 10261, 29889]])
<s>I am studying for my Master's degree in the UK.


In [108]:
encoded_prompt1

[1, 306, 626, 23382]

In [94]:
prompt1 = "I am studying"
prompt2 = "My dream is"

encoded_prompt1 = tokenizer.encode(prompt1, return_tensors="pt")
encoded_prompt2 = tokenizer.encode(prompt2, return_tensors="pt")

inputs_batch = torch.cat((encoded_prompt1, encoded_prompt2), dim=0)

# Generate
generated_ids = model.generate(inputs_batch, max_length=30)
id_to_token = tokenizer.batch_decode(generated_ids)
id_to_token


["<s>I am studying for my Computer Science class and don't understand how to answer this question. Please help to explain how to solve this problem.",
 '<s>My dream is to be a professional soccer player. I have been playing soccer since I was 5 years old. I have been playing']

In [95]:
inputs_batch.shape

torch.Size([2, 4])

In [114]:
model.generate(inputs=[3])

AttributeError: 'list' object has no attribute 'shape'